<a href="https://colab.research.google.com/github/AdamJelley/AdventOfCode2021/blob/main/Day16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 16

In [51]:
import requests
import numpy as np

In [2]:
#Variables to set
day = 16
cookie = ''

In [3]:
input_link = f'https://adventofcode.com/2021/day/{day}/input'
user_cookie = {'session':cookie} # Retrieve session cookie corresponding to user login (by inspecting cookies on data page)

In [4]:
def convert_hex_to_binary(input):
  binary_input = bin(int(input, 16))[2:] # Convert to binary and ignore leading 0b

  if len(binary_input) != len(input)*4: #Check for any missing leading zeros
    num_leading_zeros_required = len(input)*4 - len(binary_input)
    binary_input = '0'*num_leading_zeros_required + binary_input

  assert len(binary_input) == len(input)*4
  return binary_input

In [5]:
# Get hexadecimal input and convert to binary
raw_input = requests.get(input_link, cookies=user_cookie).text[:-1]
binary_input = convert_hex_to_binary(raw_input)
binary_input

'111000000101001001011101100110000000001011111010000000001011100000000000001000011011000100111110001011010100001001100000000000000100001100100001110111000110010010001101011100101001110111010110011110110010010000010010000000001001100101100110110101110110110000000001010110011110110100100111010011110110100100100001010000000010111010011111110101001010110000011011000011110110010100101100110100110011100111010111101110000010001001000000000010000011110010011010010101001110100000011001100000000010101100110110100111011100000000001000001011001111100100001100111110010010100000000000100000010111001001111101101011110100000111100110101001011100000110111001101110001110010000011010010011110011000110100100111011110110011111100010000000001001100000110100000000010101100110000110111110011010101111100100000111100111110101100000100000000010000100111001001100011100101100000000010000100111000011011110010111011101010011000000000100001110000000001101010100000100000000011011100010100111000010001110001111111000000

## Part 1

In [116]:
class Decoder:
  def __init__(self):
    self.version_sum=0
    self.operators = {0: lambda x: sum(x),
             1: lambda x: np.prod(x),
             2: lambda x: min(x),
             3: lambda x: max(x),
             5: lambda x: 1 if x[0]>x[1] else 0,
             6: lambda x: 1 if x[0]<x[1] else 0,
             7: lambda x: 1 if x[0]==x[1] else 0}

  def decode_packet_metadata(self, input):
    packet_version = int(input[:3],2)
    packet_type = int(input[3:6], 2)
    #print(f'Version: {packet_version}, Type: {packet_type}')
    self.version_sum+=packet_version
    remainder = input[6:]
    return packet_version, packet_type, remainder

  def decode_literal(self, input):
    binary_literal = ''
    for i in range(0, len(input), 5):
      if input[i]=='1':
        binary_literal+=input[i+1:i+5]
      elif input[i]=='0':
        binary_literal+=input[i+1:i+5]
        remainder = input[i+5:]
        break
    literal = int(binary_literal, 2)
    #print(f'Literal: {literal}')
    return literal, remainder

  def decode_operator(self, operator_packet_type, input):

    length_type = input[0]
    if length_type == '0':
      length_subpackets = int(input[1:16], 2)
      subpackets = input[16:16+length_subpackets]
      remainder = input[16+length_subpackets:]
      values = []
      while len(subpackets)>10:
        packet_version, packet_type, subpackets = self.decode_packet_metadata(subpackets)
        if packet_type==4:
          literal, subpackets = self.decode_literal(subpackets)
          values.append(literal)
        else:
          result, subpackets = self.decode_operator(packet_type, subpackets)
          values.append(result)
      final_result = operators[operator_packet_type](values)

    elif length_type == '1':
      num_subpackets = int(input[1:1+11], 2)
      remainder = input[12:]
      values = []
      j=0
      while j<num_subpackets:
        packet_version, packet_type, remainder = self.decode_packet_metadata(remainder)
        j+=1
        if packet_type==4:
          literal, remainder = self.decode_literal(remainder)
          values.append(literal)
        else:
          result, remainder = self.decode_operator(packet_type, remainder)
          values.append(result)
      final_result = operators[operator_packet_type](values)

    return final_result, remainder

  def decode_packet(self, input):
    self.version_sum=0
    packet_version, packet_type, remainder = self.decode_packet_metadata(input)
    if packet_type==4:
      final_result, remainder = self.decode_literal(remainder)
    else:
      final_result, remainder = self.decode_operator(packet_type, remainder)

    return final_result, remainder

In [119]:
decoder = Decoder()

for hex_test in ['8A004A801A8002F478', '620080001611562C8802118E34', 'C0015000016115A2E0802F182340', 'A0016C880162017C3686B18A3D4780']:
  binary_test = convert_hex_to_binary(hex_test)
  decoder.decode_packet(binary_test)
  print(f'Test case version sum = {decoder.version_sum}')


Test case version sum = 16
Test case version sum = 12
Test case version sum = 23
Test case version sum = 31


In [121]:
decoder.decode_packet(binary_input)
print(f'Input packet version sum = {decoder.version_sum}')

Input packet version sum = 940


## Part 2

In [103]:
decoder = Decoder()

In [123]:
test_cases = [('C200B40A82',3),('04005AC33890',54),('880086C3E88112',7),('CE00C43D881120',9),('D8005AC2A8F0',1),('F600BC2D8F',0),('9C005AC2F8F0',0),('9C0141080250320F1802104A08',1)]

for test_case in test_cases:
  binary_test_case = convert_hex_to_binary(test_case[0])
  result, remainder = decoder.decode_packet(binary_test_case)
  print(f'Test case result = {result}, expected {test_case[1]}')
  assert result==test_case[1]

Test case result = 3, expected 3
Test case result = 54, expected 54
Test case result = 7, expected 7
Test case result = 9, expected 9
Test case result = 1, expected 1
Test case result = 0, expected 0
Test case result = 0, expected 0
Test case result = 1, expected 1


In [124]:
final_result, remainder = decoder.decode_packet(binary_input)
print(f'Final result from decoding input packet = {final_result}')

Final result from decoding input packet = 13476220616073
